In [ ]:
import codecs
import json
import random
from collections import defaultdict

class WordUtils:
    
    @staticmethod
    def get_frequency(word):
        freq = defaultdict(int)
        for char in word:
            freq[char]+=1
        return freq
    
    @staticmethod
    def get_anagram(word):
        return ''.join(random.sample(word, len(word)))
    
    @staticmethod
    def why_not_composable(innerword, fromword):
        fromfreq = WordUtils.get_frequency(fromword)
        innerfreq = WordUtils.get_frequency(innerword)
        for c,count in innerfreq.items():
            if(fromfreq[c] == 0):
                return ('no_char', str(c))
            if(fromfreq[c] < count):
                return ('too_many_char', str(c))
        return None
    
    @staticmethod
    def is_composable(innerword, fromword):
            return WordUtils.why_not_composable(innerword, fromword) is None
        

class Vocabulary:
    def __init__(self, words):
        if isinstance(words, dict):
            self.words = words
        else:
            self.words = set(words)
            
    @staticmethod
    def read_from(vocabularyfile, encoding='utf8'):
        words = []
        with codecs.open(vocabularyfile, 'r', encoding) as f:
            words = [line.strip() for line in f]
        return Vocabulary(words)
    
    @staticmethod
    def read_definitions_from(vocabularyfile, encoding='utf8',sep='|', saveas = lambda line: line[1:]):
        definitions = defaultdict(list)
        with codecs.open(vocabularyfile, 'r', encoding) as f:
            for line in f:
                line = line.strip().split(sep)
                definitions[line[0]].append(saveas(line))
        return Vocabulary(definitions)
     
    def __contains__(self, word):
        return word in self.words
    
    def __getitem__(self, word):
        return self.words[word]
    
    def get_randword(self, minlen=1):
        appropriate = [w for w in self.words if len(w) >= minlen]
        return appropriate[random.randint(0, len(appropriate)-1)]

    def get_composable_words(self, word):
        freq = WordUtils.get_frequency(word)
        innerwords = set()
        for w in self.words:
            wfreq = WordUtils.get_frequency(w)
            if all(freq[char] >= count for (char,count) in wfreq.items()):
                innerwords.add(w)
        innerwords.discard(word)
        return list(sorted(innerwords))
    
   
    
    
    

    
# Игра "Наборщик"
# Кто сможет составить наибольшее количество слов из букв заданного длинного слова 
# (количество букв имеет значение, а порядок нет)
# Пример: олицетворение - лицо, творение, тело, вор, ...
# Предлагается выводить слово, указывать общее количество скрытых в слове слов, и предлагать поиграть с друзьями в эту игру.
# Можно на "последней странице" привести список всех слов, которые были загаданы, для тех, кто захочет себя проверить.
# Ответы предоставляются (и просто так (обычные слова) и в виде анаграм (в словах перемешаны буквы), как подсказка к загадке)


# vocabularyfile - cловарь - текстовый документ, одно слово в строке, utf8
# word - конкретное слово для игры или wordfrom - минимальная длинна основного слова
# minlen - минимальная длинна искомых слов

# результат:
# word: основное слово
# answer: слова найденные в данном
# clue: подсказки (анаграммы) для всех найденных слов 

# вместе с кодом предоставляются словари существительных для русского и английского языка 
# (только распространенные или все)

def compositor_game(game_generator, word = None, wordfrom=15, minlen=3):
    
    if word is None:
        word = game_generator.get_randword(wordfrom)
    composable = [w for w in game_generator.get_composable_words(word) if len(w) >= minlen]    
    anagrams = [WordUtils.get_anagram(w) for w in composable]
    
    return {'word': word, 'answer':composable, 'clue':anagrams}

In [ ]:
fullru = Vocabulary.read_from(r'locale\ru\vocab\noun.full.ru.txt')
commru = Vocabulary.read_from(r'locale\ru\vocab\noun.ru.txt')
allru = Vocabulary.read_from(r'locale\ru\vocab\all.full.ru.txt')
defru = Vocabulary.read_definitions_from(r'locale\ru\vocab\ozhegov.hint.txt', saveas=lambda line: line[1])

In [ ]:
print(u'слово' in allru)
print(u'blabla' in defru)
print(defru[u'коса'])
print(defru.get_randword())

In [ ]:
import re
import random
import json

class Speaker:
    def __init__(self, phrases, counted={}):
        self.phrases = phrases
        self.counted = counted
        
    def speak(self, about, **kwargs):
        phrase = self.get_phrase(about) 
        for (name,value) in kwargs.items():
            if isinstance(value, int):
                # вы нашли #count# @слов 
                while True:
                    m = re.search('#'+name+'#(\s+)@(\w+)', phrase)
                    if m is None:
                        break
                    replacement = str(value)+m.group(1)+self.speak_count_of(value, m.group(2))
                    phrase = phrase[:m.start()]+replacement+phrase[m.end():]
            phrase = re.sub('#'+name+'#', str(value), phrase)
        return phrase
    
    # 1 слово/очко, 2-4 слова/очка, 0,5-10,11-14,25-100 слов/очков 
    def speak_count_of(self, count, ofobject):
        if ofobject not in self.counted:
            return ofobject
        variants = self.counted[ofobject]
        lastdigit = abs(count)%10
        prevdigit = (abs(count)%100)//10
        
        if prevdigit != 1:
            if lastdigit == 1 :
                return variants[0]
            elif lastdigit <= 4 and lastdigit >= 2 and prevdigit != 1:
                return variants[1]
        return variants[2]        
    
    def get_phrase(self, about):
        if about not in self.phrases:
            return about
        phrases_about = self.phrases[about]
        if len(phrases_about)==0:
            print("No phrases: "+about)
            return about
        phraseind = random.randint(0, len(phrases_about)-1)
        return phrases_about[phraseind]
                                   
                                   
def get_speaker(phrasesfile):
    data = None
    with codecs.open(phrasesfile, 'r', 'utf8') as f:
        data = json.loads(f.read())
    
    return Speaker(data['phrases'], data['counted'])
                

In [ ]:
speaker = get_speaker(r'locale\ru\phrases.json')
print(speaker.speak("no.too_many_char", word = "foo", task = "forest", char="o"))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=0))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=11))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=314))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=121))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=22))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=24))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=37))
#print(speaker.phrases)
#print(speaker.counted)

In [ ]:
import re
class CommandParser:
    def __init__(self, commands):
        self.commands = commands
    
    def parse(self, message):
        for (cformat, command) in self.commands:
            if re.search(cformat, message, flags=re.IGNORECASE):
                return command
        return None

def get_command_parser(commandsfile):
    commands = None
    with codecs.open(commandsfile, 'r', 'utf8') as f:
        commands = json.loads(f.read())
    return CommandParser(commands)
            

In [ ]:
command_parser = get_command_parser(r'locale\ru\commands.json')
for (cformat, command) in command_parser.commands:
    print (cformat)
    print(command)
    
msgs = [u'Давай Играть еще раз!', u'сыграем ещё раз?', u'какой счет?', u'сколько у нас очков?', u'какое было задание?', u'какое слово?']
for msg in msgs:
    print(msg)
    print(command_parser.parse(msg))

In [ ]:
class Game:
    def __init__(self, word, composable_words_factors, defs, minlen, checkword):
        '''composable_words_factors - all words with their score-factors: score += len(word)*factor(word)
        checkword - check the correctness word in language (return 2 if word doesn't exist) 
        and check that the word is noun (return 1 if word exists but isn't noun) and return 0 if word is noun
        defs - is definitions hints for some words'''
        self.word = word
        self.composable_words = composable_words_factors
        self.minlen = minlen
        self.checkword = checkword
        self.definitions = defs
        
def get_new_game():
    word = fullru.get_randword(15)
    minlen = 3
    commonwords = [w for w in commru.get_composable_words(word) if len(w) >= minlen]
    rarewords = [w for w in fullru.get_composable_words(word) if len(w) >= minlen]
    scores = {}
    for w in rarewords:
        scores[w] = 3
    for w in commonwords:
        scores[w] = 1
    checkword = lambda w: 0 if w in commru or w in fullru else (1 if w in allru or w in defru else 2)
    return Game(word, scores, defru, minlen=minlen, checkword = checkword)

In [ ]:
import re 
from collections import deque

class PlayerScore:
    def __init__(self, name):
        self.name = name
        self.score = 0
        self.words = dict()       
    
class WordGame:
    def __init__(self, get_new_game, speaker, commands=None):
        '''get_new_game is function that should return object game with word 
        and dictionary with composable words and theirs score factor'''
        
        self.get_new_game = get_new_game
        self.speaker = speaker
        self.commands = commands
        self.game = None
        self.status = "NotStarted"
        self.players = dict() #score and words for each player by id
        self.foundwords = dict() #what word was mentioned by what user
        
        
        self.message_too_long_was_said = 0
        self.recently_hinted = deque()
        
    
    def register_player(self, player_id, name):
        if player_id not in self.players:
            self.players[player_id] = PlayerScore(name)
        
    def process(self, player_id, message):
        if self.commands is not None:
            command = self.commands.parse(message)
            if command is not None:
                message = command
        
        if message.startswith('/help'):
            return self.get_help()
        if message.startswith('/start') or self.status=="NotStarted": 
            return self.start_new_game()
        if message.startswith("/done") or len(self.foundwords)==len(self.game.composable_words) or self.status == "GameOver":
            return self.end_game()
        if message.startswith('/hint'):
            return self.get_hint()
        if message.startswith("/score"):
            return self.get_score()
        if message.startswith("/task"):
            return self.get_task()
        
        if len(message) > 150:
            return self.get_message_too_long()

        return self.process_words(player_id, message)

    # сообщение + корректное или некорректное слово
    def process_word(self, player_id, word):
        word = word.lower()
        
        
        if word in self.foundwords:
            return {"phrase": self.speaker.speak('no.word_already_said', word=word), "word": word, "correct": True}
        if word in self.game.composable_words:
            wfactor = self.game.composable_words[word]
            wscore = wfactor * (len(word) + sum(range(0,max(0,len(word)-4))))
            player_score = self.players[player_id]
            player_score.words[word] = wfactor
            player_score.score += wscore
            self.foundwords[word] = player_id
            return {"phrase": self.speaker.speak('ok.rare_word' if wfactor > 1 else 'ok', word=word, wscore=wscore), 
                    "word":word, "correct":True}
        
        wordstatus = self.game.checkword(word)
        if wordstatus == 2:
            return {"phrase": self.speaker.speak('no.word_not_exist', word=word), "word": word, "correct": False}
        if wordstatus == 1:
            return {"phrase": self.speaker.speak('no.word_is_not_noun', word=word), "word": word, "correct": False}
        if len(word) < self.game.minlen:
            return {"phrase": self.speaker.speak('no.too_short', word=word), "word": word, "correct": False}
        
        #это существующее слово и оно существительное. возможно,  
        
        why_wrong = WordUtils.why_not_composable(word, self.game.word)
        if why_wrong is not None:
            return {"phrase":self.speaker.speak('no.'+why_wrong[0], word = word, taskword = self.game.word, char=why_wrong[1]), 
                    "word": word, "correct": False}
        else:
            return {"phrase": "FATAL ERROR! YOU BREAK ME COMPLETELY!", "word": word, "correct": False}
        
    def process_words(self, player_id, message):
        message = message.lower()
        words = re.findall(r'[\w-]+', message)
        if (len(self.players) > 1 and re.search(r'[^\s,\.а-яё-]', message)) or len(words) == 0:
            return ""
        
        if len(words)==1:
            return self.process_word(player_id, words[0])["phrase"]
        
        if (len(self.players) > 1): #многопользовательская игра - человек может переписываться с другими игроками
            nnouns = [self.game.checkword(w) for w in words].count(0)
            if nnouns/len(words) < 0.5:
                return ""
        
        results = [self.process_word(player_id, w) for w in words]
        ncorrect = len([r for r in results if r["correct"]])
        if ncorrect > 0 or len(results) <= 3:
            return '\n'.join(r["phrase"] if r["word"] in r["phrase"] else r["word"]+". "+r["phrase"] for r in results)
        else:
            return self.speaker.speak('no.all_is_wrong')            
    
    def get_message_too_long(self):
        if self.message_too_long_was_said==0 or len(self.players) == 1 or random.randint(0,self.message_too_long_was_said**2) == 0:
            self.message_too_long_was_said += 1
            return self.speaker.speak('message.too_long')
        return ""
      
    def start_new_game(self):
        self.game = self.get_new_game()
        self.players = dict()
        self.foundwords = dict()
        self.status = "InGame"
        return self.speaker.speak('new_game', taskword=self.game.word, count=len(self.game.composable_words))
    
    def end_game(self):
        self.status = "GameOver"
        return self.speaker.speak('gameover')+'\n\n'+self.get_score(byplayers=True, scoretype="gameover")
    
    def get_task(self):
        return self.speaker.speak('what_task', taskword=self.game.word, count = len(self.game.composable_words))
    
    def get_score(self, byplayers=None, scoretype='ingame'):
        if byplayers is None:
            byplayers = random.randint(0,1000)%5 == 0
        score = self.speaker.speak('score.'+scoretype, count=len(self.foundwords), score=sum([p.score for p in self.players.values()]))
        if byplayers and len(self.players) > 1:
            for player in sorted(self.players.values(), key= lambda p: -p.score):
                score += "\n"+self.speaker.speak('score.player', name=player.name, score=player.score, count=len(player.words))
        return score
    
    #hint type: anagram, word 
    def get_hint(self):
        notfound = [w for w in self.game.composable_words.keys() if w not in self.foundwords]
        for word in sorted(self.game.composable_words, 
                          key=lambda w: abs(len(w)-9)+self.game.composable_words[w] + random.randint(0,5) + (4 if w in self.recently_hinted else 0)):
            count = len([w for w in notfound if WordUtils.is_composable(w, word)])
            if count == 0:
                continue
            
            self.recently_hinted.append(word)
            if len(self.recently_hinted) > 50:
                self.recently_hinted.popleft()
        
            if word in self.foundwords:
                if random.randint(0,2)==0:
                    return self.speaker.speak('hint.anagram', hint = WordUtils.get_anagram(word), count=count)
                else:
                    return self.speaker.speak('hint.word', hint=word, count=count)
            
            if word not in self.game.definitions or random.randint(0,4)==0:
                #anagram
                return self.speaker.speak('hint.anagram', hint = WordUtils.get_anagram(word), count=count)
            else:
                #hint with definition
                defs = self.game.definitions[word]
                return self.speaker.speak('hint.definition', hint=defs[random.randint(0,len(defs)-1)], count=count)
        return ""
    
    def get_help(self):
        '''
        Основные правила:
        Собираем новые слова из букв загаданного слова. Только существительные.

        Основные команды:

        /start - начать новую игру
        
        По-русски можно написать что-то вроде:
        сыграем еще раз
        Хочу играть ещё раз!
        задай слово
        Загадай новое слово
        
        /done - завершить игру
        
        я закончил игру
        заверши игру
        
        /task - показать слово из текущей игры
        
        Какое было загадано слово?
        какое задание
        покажи/напиши слово
        Напомни-ка мне, пожалуйста, с каким словом мы играем?

        /score - показать текущий счет
        
        какой счет
        выведи/покажи/напомни счет
        
        /hint - попросить подсказку
        
        покажи подсказку
        дай подсказку
        подскажи слово        

        /help  - показать данную справку
        
        Покажи справку
        Как играть?
        Что делать?
        '''
        return self.get_help.__doc__
        

In [ ]:
def get_test_game(word):
    minlen = 3
    commonwords = [w for w in commru.get_composable_words(word) if len(w) >= minlen]
    rarewords = [w for w in fullru.get_composable_words(word) if len(w) >= minlen]
    scores = {}
    for w in rarewords:
        scores[w] = 3
    for w in commonwords:
        scores[w] = 1
    checkword = lambda w: 0 if w in commru or w in fullru else (1 if w in allru or w in defru else 2)
    return Game(word, scores, defru, minlen=minlen, checkword = checkword)


game = WordGame(lambda: get_test_game(u'гидроэлектростанция'), speaker, command_parser)
msgs = [('Ksenia', [u'/start', u'/score', u'станция, рост, рок, трос, стон', u'Какой счет']),
        ('Alexey', [u'Привет, как у тебя дела?']),
        ('Ksenia', [u'Отлично присоединяйся', u'тяга дрон трон', u'скажи слово']),
        ('Alexey', [u'электро танец цена', u'электрон', u'торнадо', u'документ']),
        ('Ksenia', [ 
                    u'Я собираюсь написать чертовски длинное сообщение про бычка Идет бычок шатается, вздыхает на ходу, ой доска кончается, сейчас я упаду Надеюсь оно много-много длиннее чем 150 символов, иначе ничего не получится', 
                    u'Да я люблю писать длинные сообщения. Про бычков, про девочек Тань, про Цветик-семицветик могу, но лучше не буду, потому что все равно никто читать не будет. Что ты на это скжешь, а. Вопрос ставить не буду а то сразу догадается что сообщение не ему',   
                    u'И еще одно чертовски длинное сообщение про нашу Таню Наша Таня громко плачет, уронила в речку мячек, тише Танечка не плач Не утонет в речке мяч. А вот ты туда не лезь, потому что ты не мяч и утонуть можешь преспокойненько',
                    u'Вот здесь можно посмотреть слова http:\\google.ru',
                    u'Чего молчишь?']),
        ('Alexey', [u'Да, я знаю. А какое у нас слово ты не помнишь?', u'трос торс рост мост']),
        ('Ksenia', [u'Может еще одно чертовски длинное сообщение про бычка написать Идет бычок шатается, вздыхает на ходу, ой доска кончается, сейчас я упаду Надеюсь оно много-много длиннее чем 150 символов, иначе ничего не получится', 
                    u'Вообще робот сейчас уже должен замолкнуть. Про бычков, про девочек Тань, про Цветик-семицветик могу, но лучше не буду, потому что все равно никто читать не будет. Что ты на это скжешь, а. Вопрос ставить не буду а то сразу догадается что сообщение не ему',   
                    u'И не реагировать на еще одно чертовски длинное сообщение про нашу Таню Наша Таня громко плачет, уронила в речку мячек, тише Танечка не плач Не утонет в речке мяч. А вот ты туда не лезь, потому что ты не мяч и утонуть можешь преспокойненько',
                    ]),
        ('Alexey', [u'Дай подсказку', u'/hint', u'/hint', u'подскажи слово']),
        ('Ksenia', [u'/done'])]
for (player, pmsgs) in msgs:
    for msg in pmsgs:
        if player not in game.players:
            game.register_player(player, player)
        print(player+": "+msg)
        print(game.process(player, msg))

In [ ]:
import time
class Log:
    def __init__(self, filename, step=100):
        self.filename = filename
        self.file = codecs.open('telegram.log.txt', 'a', 'utf8')
        self.lastflush = time.time()
    
    def __call__(self, message):
        if time.time() - self.lastflush >= 60:
            self.file.close()
            self.file = codecs.open('telegram.log.txt', 'a', 'utf8')
            self.lastflush = time.time()
        self.file.write(time.strftime("%d %b %Y %H:%M:%S", time.localtime()) + ' '+message)

In [ ]:
import telegram
import time
from collections import defaultdict

token = open("bot.token.txt").read().strip()

bot = telegram.Bot(token)
print("me: "+bot.getMe().first_name)


last_update_id = 0
last_message_inchat = {}
games = {}

speaker = get_speaker(r'locale\ru\phrases.json')
command_parser = get_command_parser(r'locale\ru\commands.json')

updates = bot.getUpdates()
last_update_id = max([u.update_id for u in bot.getUpdates(limit=2000)]+[0])
print(last_update_id)

log = Log('telegram.log.txt')

while True:
    try:
        updates = bot.getUpdates(offset=last_update_id+1)
    except Exception as e:
        log(str(e))
    for u in updates:   
        if u.update_id > last_update_id:
            last_update_id = u.update_id
            chat_id = u.message.chat.id
            user_id = u.message.from_user.id
            user_name = u.message.from_user.first_name
            
            if chat_id not in last_message_inchat:
                last_message_inchat[chat_id] = 0
                games[chat_id] = WordGame(get_new_game, speaker, command_parser)
                log('OPEN NEW CHAT: '+str(chat_id)+'\n')
                print('OPEN NEW CHAT: '+str(chat_id)+'\n')
             
            game = games[chat_id]
            if u.message.message_id <= last_message_inchat[chat_id]:
                continue
            log('get from ('+str(user_id)+' '+str(user_name)+') in chat '+str(chat_id)+' message: '+u.message.text+'\n')
            last_message_inchat[chat_id] = u.message.message_id
            
            if user_id not in game.players:
                game.register_player(user_id, user_name);
            answer = game.process(user_id, u.message.text)
            try:
                bot.sendMessage(chat_id, answer)
                log('send to '+str(chat_id)+' message: '+answer+'\n')
            except Exception as e:
                log(str(e))
                bot = telegram.Bot(token)
    time.sleep(1)
    


In [ ]:
with codecs.open('full.ru.txt', 'w+', 'utf8') as f:
    for w in commru.words | fullru.words | allru.words:
        f.write(w+'\n')